In [ ]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [ ]:
# data=pd.read_csv('400sentences.csv',sep=";") 
#
data = pd.read_csv('sentiment_tweets3.csv') #Testing with some other dataset to see if the problem is with data or model.
# data['ans'] = data['ans'].str.lower()
data.rename(columns = {"message to examine" : "ans" , "label (depression result)" : "issp"}, inplace = True)
data.drop(['Index'], axis=1, inplace = True)
print(data.head())

                                                 ans  issp
0  just had a real good moment. i missssssssss hi...     0
1         is reading manga  http://plurk.com/p/mzp1e     0
2  @comeagainjen http://twitpic.com/2y2lx - http:...     0
3  @lapcat Need to send 'em to my accountant tomo...     0
4      ADD ME ON MYSPACE!!!  myspace.com/LookThunder     0


In [ ]:
# stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
#              "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
#              "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
#              "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
#              "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
#              "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
#              "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
#              "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
#              "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
#              "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
#              "your", "yours", "yourself", "yourselves" ]
stopwords = []

In [ ]:
def remove_stopwords(data):
  data['ans without stopwords'] = data['ans'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_ans']= data_without_stopwords['ans without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_ans'] = data_without_stopwords['clean_ans'].str.replace('[{}]'.format(string.punctuation), ' ')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
ans_list = []
cleanans = data_without_stopwords['clean_ans']
for i in range(len(data_without_stopwords['clean_ans'])):
  ans_list.append(cleanans[i])
 
issp = data_without_stopwords['issp']
#Checking
for i in range(int(len(cleanans)/40)) :
  print(cleanans[i]," --- ", issp[i])

just had a real good moment  i missssssssss him so much   ---  0
is reading manga http   plurk com p mzp1e  ---  0
 comeagainjen http   twitpic com 2y2lx   http   www youtube com watch v zoGfqvh2ME8  ---  0
 lapcat Need to send  em to my accountant tomorrow  Oddly  I wasn t even referring to my taxes  Those are supporting evidence  though   ---  0
ADD ME ON MYSPACE    myspace com LookThunder  ---  0
so sleepy  good times tonight though  ---  0
 SilkCharm re   nbn as someone already said  does fiber to the home mean we will all at least be regular now  ---  0
23 or 24ï¿½C possible today  Nice  ---  0
nite twitterville workout in the am  ciao  ---  0
 daNanner Night  darlin   Sweet dreams to you  ---  0
Good morning everybody   ---  0
Finally  I just created my WordPress Blog  There s already a blog up on the Seattle Coffee Community     http   tinyurl com c5uufd  ---  0
kisha they cnt get over u til they get out frm under u just remember ur on top  ---  0
 nicolerichie Yes i remember th

In [ ]:
X_train, X_test,Y_train, Y_test = train_test_split(cleanans, issp, test_size=0.2, random_state = 45)

In [ ]:
print(X_test, "\n", Y_test)

7701          is going to get contact lenses    hopefully
3884    aches after working out    off to my brothers ...
2959     ScruffyPanther your like me too many baby gen...
1059    Old enough to know better  young enough to not...
4235    all of my friends are busy with their test s  ...
                              ...                        
6775     ARobotsDeath I d love that superpower  do you...
866           pedrocs aahh cuddy      power truly is sexy
4701                  CruncyK Night  Kevin  Sweet Dreams 
9099    My mom told me about her depression and how sh...
8998    Try to keep up with social activities even if ...
Name: clean_ans, Length: 2063, dtype: object 
 7701    0
3884    0
2959    0
1059    0
4235    0
       ..
6775    0
866     0
4701    0
9099    1
8998    1
Name: issp, Length: 2063, dtype: int64


In [ ]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
#Mapping of the words to their index. important later for glove.
words_to_index = tokenizer.word_index

In [ ]:
def read_glove_vector(glove_vec):
  #Code copied to return word_to_vec_map. similar to Seq Model course on coursera.
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
word_to_vec_map = read_glove_vector('gdrive/MyDrive/glove.6B.50d.txt')

In [ ]:
maxLen = 150
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0] #getting the shape by testing random word.
#Standard emb matrix W*Onehot = emb vec
emb_matrix = np.zeros((vocab_len, embed_vector_len))
#each row is going to be a new word of emb vec len.
for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None and index<vocab_len:
    emb_matrix[index, :] = embedding_vector #set that index in the emb matrix to the glove vector.

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)
# standard emb layer that takes in all words from vocab and uses the emb matrix.

In [ ]:
import tensorflow as tf
def issp_pred1(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.5)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.5)(X)

  X = LSTM(128)(X)

  X = Dropout(0.5)(X)
  
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model
def issp_pred2(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  # X = LSTM(128, return_sequences=True)(embeddings)

  bl = tf.keras.layers.Bidirectional(LSTM(128, return_sequences=True))

  X = bl(embeddings)
  X = Dropout(0.5)(X)

  x = Dense(10, activation="relu")(X)

  # X = Dropout(0.5)(X)
  X = Dropout(0.5)(X)
  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

def issp_pred3(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.5)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.5)(X)

  # X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [ ]:
model1 = issp_pred1((maxLen,))
model2 = issp_pred2((maxLen,))
model3 = issp_pred3((maxLen,))

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()
adam = tf.keras.optimizers.Adam(learning_rate = 0.007)

model1.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train_indices, Y_train, batch_size=64, epochs=20)

model2.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train_indices, Y_train, batch_size=64, epochs=20) 

model3.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
model3.fit(X_train_indices, Y_train, batch_size=256, epochs=20)

Epoch 1/20
129/129 [==============================] - 8s 28ms/step - loss: 0.5500 - accuracy: 0.7730
Epoch 2/20
129/129 [==============================] - 4s 27ms/step - loss: 0.5360 - accuracy: 0.7760
Epoch 3/20
129/129 [==============================] - 4s 27ms/step - loss: 0.5371 - accuracy: 0.7759
Epoch 4/20
129/129 [==============================] - 4s 28ms/step - loss: 0.5402 - accuracy: 0.7760
Epoch 5/20
129/129 [==============================] - 4s 27ms/step - loss: 0.5397 - accuracy: 0.7760
Epoch 6/20
129/129 [==============================] - 4s 27ms/step - loss: 0.5378 - accuracy: 0.7760
Epoch 7/20
129/129 [==============================] - 3s 27ms/step - loss: 0.5359 - accuracy: 0.7760
Epoch 8/20
129/129 [==============================] - 4s 27ms/step - loss: 0.5364 - accuracy: 0.7760
Epoch 9/20
129/129 [==============================] - 4s 27ms/step - loss: 0.5358 - accuracy: 0.7760
Epoch 10/20
129/129 [==============================] - 5s 38ms/step - loss: 0.5362 - accura

In [ ]:
X_test_indices = tokenizer.texts_to_sequences(X_test)
X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')
model1.evaluate(X_test_indices, Y_test)
model2.evaluate(X_test_indices, Y_test)
model3.evaluate(X_test_indices, Y_test)

65/65 [==============================] - 1s 9ms/step - loss: 0.0522 - accuracy: 0.9825


[0.0521932952105999, 0.982537031173706]

In [ ]:
reviews_list_idx = tokenizer.texts_to_sequences(ans_list)
def add_score_predictions(reviews_list_idx):

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model3.predict(reviews_list_idx)

  # pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  for i in range(len(ans_list)) :
    print(ans_list[i], " -- ",np.sum(review_preds[i])/len(review_preds[i]))


In [ ]:
add_score_predictions(reviews_list_idx)